# Defining Model Tester

In [1]:
from french_crs.fast_model_training import model_tester

train="balanced_Full_Soon"
test="balanced_Full_Soon"

model_parameter={
                "model_name" : "../pre-trained language models/Model_ANCOR_"+train+".model",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "columns_drop_list" : [ 
                                        "SUB_CORPUS",
                                        "FILE_NAME",
                                        "COREF_TABLE_ID",
                                        "COREF_TYPE",
                                        "COREF_ANCOR_ID",
                                        "MENTION_LEFT_ID",
                                        "MENTION_RIGHT_ID",
                                        "LEFT_CONTENT",
                                        "RIGHT_CONTENT",
                                        "LEFT_DEF",
                                        "RIGHT_DEF",
                                        "ID_DEF",
                                        "DISTANCE_MENTION",
                                        "DISTANCE_WORD",
                                        "DISTANCE_CHAR"
                                       ],
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)

# Looping over Test Files

In [15]:
import glob
import json
import pandas as pd

window_size=20
counter=0
acc=0
json_relations_positive={}
split_config_json="../datasets/split_config.json"

with open(split_config_json) as json_file:
    config_json = json.load(json_file)

sub_corpus_files_test = config_json["sub_corpus_files_test"]


for sub_corpus in ["corpus_OTG","corpus_UBS","corpus_ESLO","corpus_ESLO_CO2"]:

    ancor_subcorpus_path = "../DISTRIB_ANCOR/"+sub_corpus+'/'

    corpus_files = [file[file.rfind(
        "/")+1:-4] for file in glob.glob(ancor_subcorpus_path+"annotation_integree/*.xml")]

    set1=set(corpus_files)
    set2=set(sub_corpus_files_test)
    corpus_files=set1.intersection(set2)
    
#     "1AG0141"

    list_dataframes = []

    for subfile in corpus_files:

        root_xml,root_aa,data_source=set_file_config(sub_corpus, subfile)
        json_mentions=generate_json_mentions(root_xml,root_aa)
        json_mentions=remove_empty_json_mentions(json_mentions)
        json_relations=generate_json_relations()
        if json_relations == {}:
            continue
        json_coreference_chains=generate_json_chains()
        
        df_mentions = pd.DataFrame.from_dict(json_mentions, orient='index')
        df_mentions = df_mentions.sort_values(by=['START_ID'])
        list_mentions_sorted = list(df_mentions.index)
        list_mentions_sorted_rev=list(reversed(list_mentions_sorted))
        
        print("Starting",subfile,"at",sub_corpus)
        
        for index, right_mention in enumerate(list_mentions_sorted_rev):
            
            left_mention_candidates=list_mentions_sorted_rev[index+1:index+window_size]
            
            if left_mention_candidates!=[]:
                for left_mention_candidate in left_mention_candidates:
                    
                    
                    coref_features=generate_coref_features(left_mention_candidate,right_mention,
                                                           json_relations,json_mentions,
                                                           data_source)
                    pred=model.apply_model_to_row(coref_features)
                    
                    
                    if pred>=0.7480658909510631:
#                         print(left_mention_candidate,right_mention,"\n\n")
                        counter+=1
                        acc+=pred
                        print(acc/counter)
                        json_relations_positive[counter]=coref_features
                        break

df_coref_relations = pd.DataFrame.from_dict(json_relations_positive, orient='index')
df_coref_relations.to_excel("Dataset_Test.xlsx")
print(acc,counter,acc/counter)

Starting 1AG0499 at corpus_OTG
0.8954274265261449
0.8905060525097861
0.8888655945043332
0.90192532284394
0.8746285726266153
0.876454590271084
0.8885506668705376
0.890380156206827
0.880188790337897
0.8698642978825868
0.8768330072598581
0.8674889089026769
0.8735682313742074
0.8657922024884556
0.864416258399364
0.8691692725198759
0.8677565530276526
0.8687470044424178
0.8666596976846783
0.8699110470038048
0.8647854620180224
0.8658549098121271
0.8629332990979676
0.8638771065727783
0.8647454094496042
0.8634077589550084
0.8642291263453202
0.8649918246363241
0.8676163999199943
0.8688983284251279
0.8687152391631043
0.8679789204752598
0.8702603647245722
0.8680034044756046
0.8650853468882165
0.8656377013300571
0.863042439605525
0.8605837706033364
0.8627038426571628
0.8601703805753381
0.8597434483204263
0.8593368461728913
0.8587959801669202
0.8584293933023073
0.8579327314048367
0.8567475499824069
0.8568445153467054
0.8560508472045626
0.8582134317320419
0.857424131987878
0.856392568387205
0.8552553

0.8505105743521427
0.8505637233249137
0.8505762610627214
0.8503323595854455
0.8504137744099443
0.8502134838344491
0.8502839477358151
0.850390462774482
0.8501937964854238
0.850001113063852
0.8502490389739404
0.8500571726918602
0.8498662538975927
0.8496762755899965
0.8494822260808389
0.8492536912048007
0.8493648022693949
0.8491749546562064
0.8490153089449233
0.8490825386264373
0.8491494427404864
0.8489718692216408
0.848955419064036
0.8489099490251295
0.8487243790774964
0.8485276009997254
0.8483317621967884
0.8483488678338413
0.8481663937555237
0.8479876465215379
0.8478049289774275
0.8476250288639481
0.8474100052587559
0.8477426008113929
0.8478387888599074
0.8479201329161027
0.8480136569084299
0.8479470081517759
0.848030581334723
0.8478128977577014
0.8477471744177234
0.8477026334042362
0.8474872049965516
0.8474226808831358
0.8475033018534524
0.8475709551970081
0.8476383003248756
0.8477178815625724
0.847784588103997
0.8477202670710632
0.8476562364265399
0.8477349460421667
0.847563692861986

0.8482462315630723
0.8482314122894463
0.8482640347458967
0.8482492298530294
0.8482837744014222
0.8482494180350659
0.8482346846087717
0.8482537931181715
0.8481874189755598
0.8481212049635697
0.8481066970153197
0.8480407395015055
0.8480263631414269
0.8480588009665773
0.8479721598235598
0.8480065811144641
0.8480409200577825
0.8480174737668704
0.8479498597573457
0.8479349327936706
0.8479762046662553
0.8479620588912848
0.847868912182788
0.847919670642344
0.8478210925728012
0.8477230977740308
0.8476254629410502
0.847561319540926
0.8475466627969951
0.8475321181891059
0.8475895411650072
0.8475337036573066
0.8476345632061788
0.84767555821504
0.8477164572168548
0.8476959307876405
0.8476284246909693
0.8475610761347805
0.8475662364798888
0.847552864386101
0.847444796538156
0.847381905953368
0.8473677922649817
0.8473462265324079
0.8474508624768203
0.8475552564884133
0.8475420051621173
0.847528784404048
0.847515594108555
0.8475024341704739
0.8474512781539562
0.8474382222753822
0.8474204673737331
0.8

0.848196550866848
0.8481447965019145
0.8481011618213526
0.8480614895009441
0.8480920609054409
0.8480825272125924
0.8481206366913995
0.8481575443617105
0.8481801543327058
0.8481705811156041
0.8481577632238426
0.848235909937228
0.8482263153815681
0.8482534265727221
0.8483544373435811
0.8483051387779996
0.8482710597835166
0.8482892336666722
0.848246269703546
0.8482367197335402
0.8481715333094924
0.8481285274080238
0.8480855888611387
0.8481008720194214
0.8480914730639068
0.8480508905848451
0.8480373492887554
0.8479946493306798
0.8479853624706528
0.847997864074556
0.8479333708177451
0.8479020024196384
0.8478896123229933
0.8478772414656287
0.8478680889182668
0.8478589505609009
0.8479331878411154
0.8479240132436503
0.8478599761329224
0.8478656135908724
0.8478565124281893
0.8479225691618838
0.8478746298325802
0.8479405708260037
0.8479314399846567
0.847926539019701
0.8479999105039894
0.8480731692824647
0.8480854212031093
0.8480804171880321
0.8481010812909469
0.8480870882499502
0.848104468465054

0.8479387318990844
0.8479901288569883
0.8479738747884392
0.8479979604409877
0.848022017640314
0.8480172302855952
0.8480376350498897
0.8480107158566379
0.8479853745900378
0.8479435520838471
0.8478987502783693
0.8478917978416857
0.8478848535795307
0.8479117231626426
0.8478796528630965
0.8478380667894113
0.8478220353713181
0.8478151523540524
0.84781134853541
0.8478272945090378
0.847852356533075
0.8478197523510311
0.8478356654539773
0.8478038081745273
0.8477719880897995
0.8478321059014323
0.8478252531329278
0.8478675068356503
0.8478298397195609
0.8478387993945219
0.847905941604899
0.8479261144910385
0.847949896889173
0.847910731661504
0.8478852090684798
0.8478535447958389
0.8479098759731389
0.8479314642770673
0.8479211775926213
0.8478895658017898
0.8478803502912262
0.8478444340535175
0.8478523321478859
0.8478208331126017
0.8477778612050039
0.847797945083066
0.8478180058096838
0.8478380434248654
0.8478312740319415
0.8478275287844729
0.8478207732233065
0.847814025427153
0.84782966761329
0.84

In [16]:
from french_crs.fast_pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="Dataset_Test.xlsx", 
                            path_model_file="Dataset_Test.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="IS_CO_REF",
                            scorch_output_path="./",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

SCORCH json formats saved at following addresses:
/Users/mehdi.mirzapour/French-CRS/demo/coref_chains_gold.json
/Users/mehdi.mirzapour/French-CRS/demo/coref_chains_pred.json


In [ ]:
import joblib
import numpy as np

model1 = joblib.load(model.model_name)

df_coref_relations = pd.DataFrame.from_dict(coref_features, orient='index')

input_dataframe = df_coref_relations.drop(model.columns_drop_list, axis=0)

input_dataframe.fillna("VOID", inplace=True)
input_dataframe = input_dataframe.replace(
                        model.values_list, model.values_list_for_replace)

y_pred = model1.predict(np.array(input_dataframe[0:-1]).reshape(1, -1))[0]
y_pred

In [ ]:
json_relations_positive

In [ ]:
y_pred

In [2]:
def generate_coref_features(pair_left_id,pair_right_id,
                            json_relations,json_mentions,
                            data_source):

    relation_type="CO_REFERENCE"
    relation_left=json_mentions[pair_left_id]
    relation_right=json_mentions[pair_right_id]

    if relation_left["START_ID"] > relation_right["START_ID"]:
        relation_left,relation_right=relation_right,relation_left

    MENTION_LEFT_ID=pair_left_id
    MENTION_RIGHT_ID=pair_right_id

    string_text = data_source[relation_left["END_ID"]:relation_right["START_ID"]]
    string_text=re.sub("<[a-zA-Z0-9\s=\",/.]+>", " ", string_text).strip().split()
    DISTANCE_MENTION = int(relation_right["NUM"])-int(relation_left["NUM"])-1
    DISTANCE_WORD = len(string_text)
    DISTANCE_CHAR = len("".join(string_text))


    tf2yn = {True: "YES", False: "NO"}

    left_content = str(relation_left["CONTENT"])
    right_content = str(relation_right["CONTENT"])
    left_content_list = left_content.lower().split()
    right_content_list = right_content.lower().split()
    len_of_words_min = min(len(left_content_list), len(right_content_list))
    len_of_words_max = max(len(left_content_list), len(right_content_list))
    len_intersection = len(
        [value for value in left_content_list if value in right_content_list])


    ID_FORM = tf2yn[relation_left["CONTENT"]==relation_right["CONTENT"]]
    ID_SUBFORM = tf2yn[len_intersection>0]

    INCL_RATE = len_intersection/len_of_words_min
    COM_RATE = len_intersection/len_of_words_max

    ID_DEF = tf2yn[relation_left["DEF"]==relation_right["DEF"]]
    if relation_left["DEF"] == "UNK" or relation_right["DEF"] == "UNK":
        ID_DEF = "UNK"

    ID_GP = tf2yn[relation_left["GP"]==relation_right["GP"]]
    if relation_left["GP"] == "UNK" or relation_right["GP"] == "UNK":
        ID_GP = "UNK"

    ID_TYPE = tf2yn[relation_left["TYPE"]==relation_right["TYPE"]]
    if relation_left["TYPE"] == "UNK" or relation_right["TYPE"] == "UNK":
        ID_TYPE = "UNK"

    ID_EN = tf2yn[relation_left["EN"]==relation_right["EN"]]
    if relation_left["EN"] == "UNK" or relation_right["EN"] == "UNK":
        ID_EN = "UNK"

    ID_GENDER = tf2yn[relation_left["GENRE"]==relation_right["GENRE"]]
    if relation_left["GENRE"] == "UNK" or relation_right["GENRE"] == "UNK":
        ID_GENDER = "UNK"

    ID_NUMBER = tf2yn[relation_left["NUM"]==relation_right["NUM"]]
    if relation_left["NUM"] == "UNK" or relation_right["NUM"] == "UNK":
        ID_NUMBER = "UNK"

    EMBEDDED = tf2yn[((left_content in right_content) or (right_content in left_content))]

    ID_PREVIOUS = tf2yn[relation_left["PREVIOUS"]==relation_right["PREVIOUS"]]
    if relation_left["PREVIOUS"] == "^" or relation_right["PREVIOUS"] == "^":
        ID_PREVIOUS = "UNK"

    ID_NEXT = tf2yn[relation_left["NEXT"]==relation_right["NEXT"]]
    if relation_left["NEXT"] == "^" or relation_right["NEXT"] == "^":
        ID_NEXT = "UNK"
        
    coref_relation_json_feature={
        
                                "SUB_CORPUS":sub_corpus,
                                "FILE_NAME":subfile,
                                "COREF_TABLE_ID":-1,
                                "COREF_TYPE":relation_type,
                                "COREF_ANCOR_ID":"",
                                "MENTION_LEFT_ID":MENTION_LEFT_ID,
                                "MENTION_RIGHT_ID":MENTION_RIGHT_ID,
                                "LEFT_CONTENT":relation_left["CONTENT"],                                     
                                "RIGHT_CONTENT":relation_right["CONTENT"],                                     
                                "LEFT_TYPE":relation_left["TYPE"], 
                                "RIGHT_TYPE":relation_right["TYPE"], 
                                "LEFT_DEF":relation_left["DEF"],
                                "RIGHT_DEF":relation_right["DEF"], 
                                "LEFT_GP":relation_left["GP"],
                                "RIGHT_GP":relation_right["GP"],
                                "LEFT_GENRE":relation_left["GENRE"],
                                "RIGHT_GENRE":relation_right["GENRE"],
                                "LEFT_NB":relation_left["NB"],
                                "RIGHT_NB":relation_right["NB"],
                                "LEFT_EN":relation_left["EN"],
                                "RIGHT_EN":relation_right["EN"],
                                "ID_FORM": ID_FORM, 
                                "ID_SUBFORM": ID_SUBFORM, 
                                "INCL_RATE": INCL_RATE, 
                                "COM_RATE": COM_RATE, 
                                "ID_DEF": ID_DEF, 
                                "ID_GP": ID_GP, 
                                "ID_TYPE": ID_TYPE,
                                "ID_EN": ID_EN, 
                                "ID_GENDER": ID_GENDER, 
                                "ID_NUMBER": ID_NUMBER, 
                                "DISTANCE_MENTION": DISTANCE_MENTION, 
                                "DISTANCE_WORD": DISTANCE_WORD,
                                "DISTANCE_CHAR": DISTANCE_CHAR, 
                                "EMBEDDED": EMBEDDED, 
                                "ID_PREVIOUS": ID_PREVIOUS, 
                                "ID_NEXT": ID_NEXT, 
                                "IS_CO_REF": 1
    }
    
    return(coref_relation_json_feature)
                    

In [3]:
import xml.etree.ElementTree as ET
import json
import pandas as pd
import re
import random
import glob
import networkx as nx
import os


def set_file_config(sub_corpus, file):

    path_file = "../DISTRIB_ANCOR/"+sub_corpus+'/'

    tree_xml = ET.parse(
        path_file+'annotation_integree/'+file+'.xml')
    tree_aa = ET.parse(
        path_file+'aa_fichiers/'+file+'.aa')
    data_source = open(
        path_file+'ac_fichiers/'+file + '.ac', "r").read()

    root_xml = tree_xml.getroot()
    root_aa = tree_aa.getroot()
    
    return(root_xml,root_aa,data_source)
    

# return mentions in json format
def generate_json_mentions(root_xml,root_aa):

    json_mentions = {}

    for unit in root_xml.iter("unit"):
        data_character = {}
        data_character["TYPE"] = unit.find("./characterisation/type").text

        for anchor in root_xml.iter("anchor"):
            if anchor.attrib["id"] == unit.attrib["id"]:
                data_character["NUM"] = anchor.attrib["num"]

        for feat in unit.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        for unit_aa in root_aa.iter("unit"):
            if unit_aa.attrib["id"] == unit.attrib["id"]:
                data_character["START_ID"] = int(unit_aa.find(
                    "./positioning/start/singlePosition").attrib["index"])
                data_character["END_ID"] = int(unit_aa.find(
                    "./positioning/end/singlePosition").attrib["index"])

        json_mentions[unit.attrib["id"]] = data_character

    return(json_mentions)

# return mentions in json format and eliminates all the features that are not presented.
def remove_empty_json_mentions(json_mentions,features_to_check=["CONTENT", "PREVIOUS"]):

    json_mentions_= json_mentions.copy()
    for mention in list(json_mentions_):
        for feature in features_to_check:
            if feature not in json_mentions_[mention]:
                del json_mentions_[mention]
                break

    return(json_mentions_)


# return relations in json format based on the json mentions
def generate_json_relations():

    json_relations={}

    for relation in root_aa.iter("relation"):
        data_character = {}
        data_character["TYPE"] = relation.find("./characterisation/type").text

        for feat in relation.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        unit_ids = []
        for feat in relation.findall("./positioning/term"):
            unit_ids.append(feat.attrib["id"])

        if (unit_ids[0] in json_mentions.keys()) and (unit_ids[1] in json_mentions.keys()):
            if (len(unit_ids) == 2):
                data_character["LEFT_UNIT"] = {
                    "ID": unit_ids[0], **json_mentions[unit_ids[0]]}
                data_character["RIGHT_UNIT"] = {
                    "ID": unit_ids[1], **json_mentions[unit_ids[1]]}

            json_relations[relation.attrib["id"]] = data_character

    return(json_relations)


# return chains in json format based on the json mentions and relations.
def generate_json_chains(coreference_type=["DIRECTE", "INDIRECTE", "ANAPHORE"]):
    G = nx.Graph()
    chain_id = 0
    json_coreference_chains = {}

    for chain in json_relations:
        if json_relations[chain]["TYPE"].strip() in coreference_type:
            chain_left = json_relations[chain]["LEFT_UNIT"]["ID"]
            chain_right = json_relations[chain]["RIGHT_UNIT"]["ID"]
            G.add_edge(chain_left, chain_right,
                       coref_type=json_relations[chain]["TYPE"].strip())

    for subgraph in list(nx.connected_components(G)):
        json_coreference_chains[chain_id] = []
        for node in subgraph:
            json_coreference_chains[chain_id].append(node)
        chain_id += 1

    return(json_coreference_chains)

# Dataset Splitting

In [ ]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.fast_model_training import dataset_splitter

# ds1=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
#                     "./Dataset_ANCOR_Balanced_Train.xlsx",
#                     "./Dataset_ANCOR_Balanced_Test.xlsx",
#                       split_config_json="./split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.50, files_num=68)


ds2=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
                    "./Dataset_ANCOR_Balanced_Train.xlsx",
                    "./Dataset_ANCOR_Balanced_Test.xlsx",
                      split_config_json="./split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


# Model Training

In [ ]:
from french_crs.model_training import model_trainer

"""
12 possibilities for "train" and "test" variables

balanced_ANAPHORE
balanced_DIRECTE
balanced_Full
balanced_INDIRECTE
representative_ANAPHORE
representative_DIRECTE
representative_Full
representative_INDIRECTE
window_ANAPHORE
window_DIRECTE
window_Full
window_INDIRECTE
"""

train="balanced_Full_Soon"
test="balanced_Full_Soon"

model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                     "../datasets/"+ test+"/"+test+"_Test.xlsx",
                     "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ["SUB_CORPUS",
                            "FILE_NAME",
                            "COREF_TABLE_ID",
                            "COREF_TYPE",
                            "COREF_ANCOR_ID",
                            "MENTION_LEFT_ID",
                            "MENTION_RIGHT_ID",
                            "LEFT_CONTENT",
                            "RIGHT_CONTENT",
                            "LEFT_DEF",
                            "RIGHT_DEF",
                            "ID_DEF",
                            "DISTANCE_MENTION",
                            "DISTANCE_WORD",
                            "DISTANCE_CHAR"]

model.convert_columns_to_numeric()
print("Model_Name :","Model_ANCOR_"+train+".model\n")
print("Train Dataset :",train+"_Train.xlsx")
print("Test  Dataset :",test+"_Test.xlsx")
performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)
performance

In [ ]:
train="balanced_Full_Soon"
test="balanced_Full_Soon"

# SCORCH Chains Building

In [ ]:
from french_crs.fast_pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                            path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="Prediction",
                            scorch_output_path="../",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

# SCORCH Outcome

In [ ]:
import os
bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../mm.txt"
os.system(bashCommand)
f = open("../mm.txt",'r')
message = f.read()
print(message)

# Putting All Chains Together

In [ ]:
import docx
import os
from french_crs.model_training import model_trainer
from french_crs.pairs2chains import chains_builder

mydoc = docx.Document()

style = mydoc.styles['Normal']
font = style.font
font.name = 'MS Gothic'
font.size = docx.shared.Pt(10)


train_test_list=[
    "balanced_ANAPHORE",
    "balanced_DIRECTE",
    "balanced_Full",
    "balanced_INDIRECTE",
    "representative_ANAPHORE",
    "representative_DIRECTE",
    "representative_Full",
    "representative_INDIRECTE",
    "window_ANAPHORE",
    "window_DIRECTE",
    "window_Full",
    "window_INDIRECTE"
    ]

# train_test_list=[
#     ["balanced_ANAPHORE","window_ANAPHORE"],
#     ["balanced_DIRECTE","window_DIRECTE"],
#     ["balanced_Full","window_Full"],
#     ["balanced_INDIRECTE","window_INDIRECTE"],
#     ["representative_ANAPHORE","window_ANAPHORE"],
#     ["representative_DIRECTE","window_DIRECTE"],
#     ["representative_Full","window_Full"],
#     ["representative_INDIRECTE","window_INDIRECTE"],
#     ["window_ANAPHORE","window_ANAPHORE"],
#     ["window_DIRECTE","window_DIRECTE"],
#     ["window_Full","window_Full"],
#     ["window_INDIRECTE","window_INDIRECTE"]
#     ]

counter=0

# for train in train_test_list:
#     for test in train_test_list:

for train in train_test_list:
    
    test=train
    
    counter+=1

    model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                 "../datasets/"+ test+"/"+test+"_Test.xlsx",
                 "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                 "IS_CO_REF",
                 "IS_CO_REF"
               )

    model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                               'DISTANCE_MENTION','DISTANCE_WORD',
                               'DISTANCE_CHAR']

    model.convert_columns_to_numeric()
    performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)


    model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                        path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                        gold_column="IS_CO_REF",
                        model_column="Prediction",
                        scorch_output_path="../",
                        threshold=0.5)

    model_chains.generate_gold_model_json_output(mode="train")


    bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../output.txt"
    os.system(bashCommand)
    f = open("../output.txt",'r')
    message = f.read()
    f.close()


    mydoc.add_paragraph("Model_Name : "+"Model_ANCOR_"+train+".model")
    mydoc.add_paragraph("Train Dataset : "+train+"_Train.xlsx")
    mydoc.add_paragraph("Test  Dataset : "+test+"_Test.xlsx")
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(str(performance))
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(message)
    mydoc.add_page_break()

    print(counter)


mydoc.save("../pre-trained language models/Performance Analysis.docx")

#  Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()